In [2]:
import transformers
from transformers import AutoModel,AutoTokenizer
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,random_split,DataLoader

c:\users\ravic\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BertModel=AutoModel.from_pretrained('bert-base-uncased')
Tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [4]:
MAX_LENGTH=512
BATCH_SIZE=64

In [5]:
example="hello every onr"
sampleToke=Tokenizer(example,padding='max_length',max_length=MAX_LENGTH , return_tensors="pt")
sampleToke

{'input_ids': tensor([[ 101, 7592, 2296, 2006, 2099,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [6]:
BertModel(sampleToke['input_ids'],sampleToke['attention_mask'],sampleToke['token_type_ids'])

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0430,  0.1534,  0.0510,  ..., -0.0410,  0.0748,  0.2310],
         [ 0.2137,  0.2345,  1.2482,  ...,  0.0660,  0.4136,  0.3575],
         [-0.7413, -0.4890,  0.8329,  ...,  0.0636,  0.0979,  0.1505],
         ...,
         [-0.0292, -0.0037,  0.8261,  ...,  0.0028, -0.0475,  0.0599],
         [-0.1501, -0.3032,  0.5992,  ...,  0.2535,  0.0052, -0.1078],
         [-0.3415, -0.4514,  0.4232,  ...,  0.3282, -0.0235, -0.0447]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-7.5474e-01, -2.0496e-01,  2.4208e-01,  5.6124e-01, -1.2974e-01,
         -7.3692e-02,  8.1097e-01,  1.0685e-01,  2.4107e-01, -9.9956e-01,
          2.7547e-01,  2.0497e-01,  9.6183e-01, -1.5312e-01,  8.7371e-01,
         -4.5443e-01, -7.2256e-02, -4.2038e-01,  3.4187e-01, -5.3183e-01,
          5.1893e-01,  9.5401e-01,  6.2038e-01,  2.0069e-01,  3.6993e-01,
          4.1834e-01, -4.9691e-01,  8.9083e-01,  9.2674e-01,  5.963

In [165]:
train_df=pd.read_csv("../../datasets/splitDataset/train.csv")
test_df=pd.read_csv("../../datasets/splitDataset/test.csv")
valid_df=pd.read_csv("../../datasets/splitDataset/valid.csv")


In [234]:
class CustomDataset(Dataset):
    def __init__(self,df):
        self.comment=df['comment_text']
        self.label=df['label']
    def __len__(self):
        return len(self.comment)
    def __getitem__(self,index):
        comment=str(self.comment[index])
        comment=" ".join(comment.split())
        tokens=Tokenizer(comment,
                         max_length=MAX_LENGTH,
                         padding='max_length',
                         return_tensors="pt"
                        )
        return tokens.input_ids,tokens.token_type_ids,tokens.attention_mask,self.label[index]
    

In [235]:
train_ds=CustomDataset(train_df)
test_ds=CustomDataset(test_df)
valid_ds=CustomDataset(valid_df)

In [236]:
train_loader=DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=True)
test_loader=DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=True)
val_loader=DataLoader(valid_ds,batch_size=BATCH_SIZE,shuffle=True)

In [237]:
for data in train_loader:
    print(data)

[tensor([[[  101, 16824,  3548,  ...,     0,     0,     0]],

        [[  101, 27208,  1010,  ...,     0,     0,     0]],

        [[  101,  1996,  2123,  ...,     0,     0,     0]],

        ...,

        [[  101,  8398,  3658,  ...,     0,     0,     0]],

        [[  101, 10398,  1997,  ...,     0,     0,     0]],

        [[  101, 22017,  7570,  ...,     0,     0,     0]]]), tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]]), tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]]), tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [247]:
class ToxicityClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.BertModel=BertModel
        self.Dropout=nn.Dropout(0.1)
        self.fc=nn.Linear(768,1)
    def forward(self,input_ids,token_type_ids,attention_mask):
        input_ids=input_ids.view(BATCH_SIZE,MAX_LENGTH).to(device)
        token_type_ids=token_type_ids.view(BATCH_SIZE,MAX_LENGTH).to(device)
        attention_mask=attention_mask.view(BATCH_SIZE,MAX_LENGTH).to(device)
        out=self.BertModel(input_ids,token_type_ids,attention_mask)
        out=self.Dropout(out)
        out=self.fc(out)
        out=F.softmax(out,dim=1)
        return out
    

In [248]:
device= torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [249]:
model=ToxicityClassifier()
model.to(device)

ToxicityClassifier(
  (BertModel): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [ ]:
for input_ids,token_type_ids,attention_mask,target in train_loader:
    out=model(input_ids,token_type_ids,attention_mask)
    print(out)
    break;